In [1]:
import torch
import torchvision.models as models

In [2]:
# Load the pre-trained ResNet-18 model
torch_model = models.resnet50()

# Freeze the features of the model (optional but often helps with fine-tuning)
for param in torch_model.parameters():
    param.requires_grad = True

# Modify the final layer to be a binary classifier
num_ftrs = torch_model.fc.in_features
torch_model.fc = torch.nn.Linear(num_ftrs, 2)  # The output is for 2 classes: Accident and Non Accident

In [3]:
torch_model.load_state_dict(torch.load('ipex.pth'))
torch_model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [4]:
batch_size = 1
# Input to the model
x = torch.randn(batch_size, 3, 224, 224, requires_grad=True)
torch_out = torch_model(x)

# Export the model
torch.onnx.export(torch_model,               # model being run
                  x,                         # model input (or a tuple for multiple inputs)
                  "accident_detection.onnx",   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}})

============= Diagnostic Run torch.onnx.export version 2.0.1+cu117 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [6]:
from openvino.runtime import Core
import numpy as np
import cv2
import torch
core = Core()

ir_model = core.read_model(model='accident_detection.xml', weights='accident_detection.bin')
compiled_model = core.compile_model(model=ir_model, device_name="CPU")

In [7]:
input_layer = compiled_model.input(0)
output_layer = compiled_model.output(0)

print(f"Input layer shape: {input_layer.shape}")
print(f"Output layer shape: {output_layer.shape}")

Input layer shape: [1,3,224,224]
Output layer shape: [1,2]


In [8]:
path = '/home/SSK/Repos/IntelOneAPI/data/test/Accident/test12_16.jpg'
img = cv2.imread(path)

In [9]:
frame = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
frame = cv2.resize(frame, (224, 224))
frame_org = frame
rgb_range = 255
frame = frame / rgb_range
frame = torch.from_numpy(frame).float().permute(2, 0, 1).unsqueeze(0)

In [10]:
output = compiled_model(frame)[output_layer]

In [11]:
print(output)

[[ 0.07209912 -0.01591807]]
